<a href="https://colab.research.google.com/github/jiyoungemma/User-Log-data-analytics/blob/main/recommend_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 라이브러리 & 데이터 불러오기

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from datetime import datetime
%matplotlib inline
import pandas as pd
import datetime
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

In [ ]:
# RFM지수와 cart, purchase 데이터 merge한 데이터 불러오기
RFM_without_view = pd.read_csv("/content/drive/MyDrive/RFM_without_view.csv")

In [ ]:
# 불필요한 칼럼 제거
RFM_without_view.drop(columns=['field1', 'event_time','user_id.1','category_id', 'Recency','Monetary','Recency_score', 'Frequency_score', 'Monetary_score',
       'RFM_seg', 'RFM_score'], inplace=True)

In [ ]:
RFM_without_view.head()

,event_type,product_id,category_code,brand,price,user_id,user_session,main_category,subcategory,Frequency,RFM_label
0,purchase,1004856,electronics.smartphone,samsung,130.76,543272936,8187d148-3c41-46d4-b0c0-9c08cd9dc564,electronics,smartphone,33,segment1
1,purchase,1002532,electronics.smartphone,apple,642.69,551377651,3c80f0d6-e9ec-4181-8c5c-837a30be2d68,electronics,smartphone,9,segment1
2,purchase,5100816,NaN,xiaomi,29.51,514591159,0e5dfc4b-2a55-43e6-8c05-97e1f07fbb56,NaN,NaN,1,segment5
3,purchase,13800054,furniture.bathroom.toilet,santeri,54.42,555332717,1dea3ee2-2ded-42e8-8e7a-4e2ad6ae942f,furniture,toilet,4,segment2
4,purchase,4804055,electronics.audio.headphone,apple,189.91,524601178,2af9b570-0942-4dcd-8f25-4d84fba82553,electronics,headphone,1,segment5


In [ ]:
# 누적 구매횟수가 1번인 고객 수
RFM_without_view.loc[RFM_without_view['Frequency'] == 1].shape

(530997, 11)

In [ ]:
# 구매 고객수
RFM_without_view.loc[RFM_without_view['event_type'] == 'purchase'].shape

(704461, 11)

In [ ]:
# 전체 구매 고객 중 누적 구매횟수가 1번인 고객 비율
(530997 / 704461) * 100

75.37635156523925

## baseline 모델

### baseline 데이터셋

In [ ]:
# memory-out 문제를 해결하기 위해 전처리된 데이터 불러오기
recommend = pd.read_csv("/content/drive/MyDrive/df.csv")

In [ ]:
recommend = recommend.drop(columns = 'Unnamed: 0')

In [ ]:
# 카테고리 결측치 제거
non_category = recommend[recommend['category_code'].isnull()].index
recommend.drop(non_category, inplace=True)

In [ ]:
# 구매한 데이터를 통한 추천시스템 구현하기 위해 purchase 데이터만 분리
recommend = recommend.loc[recommend['event_type'] == 'purchase']

In [ ]:
recommend.shape

(569424, 11)

### computers baseline
- 제품간 추천

In [ ]:
com_base = recommend.loc[(recommend['main_category'] == 'computers')]

In [ ]:
com_base = com_base.reset_index(drop=True)

In [ ]:
com_base.drop(columns=['event_time', 'product_id','category_id','user_id','main_category','subcategory'], inplace=True)

In [ ]:
com_base.head()

,event_type,category_code,brand,price,user_session
0,purchase,computers.notebook,hp,320.35,104dc3e4-d6ff-4f00-9031-c0468e75ed4c
1,purchase,computers.peripherals.printer,epson,116.73,63de1120-9d48-45d0-b024-500d1b95d71f
2,purchase,computers.components.hdd,kingston,66.67,629a0485-36c9-4a80-9f02-4a96382c3059
3,purchase,computers.notebook,asus,879.96,4eb54277-2072-4b17-a275-dced7c3ae7ee
4,purchase,computers.notebook,NaN,431.67,76e5233a-000b-4aac-890a-f3a7c0e4c702


In [ ]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_com = tfidf.fit_transform(com_base['category_code'])

dtm_com = pd.DataFrame(dtm_com.todense(), columns=tfidf.get_feature_names())
dtm_com

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,camera,components,computers,cooler,cpu,desktop,ebooks,hdd,keyboard,memory,monitor,motherboard,mouse,notebook,peripherals,power_supply,printer,videocards
0,0.0,0.000000,0.534712,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.845034,0.000000,0.0,0.000000,0.0000
1,0.0,0.000000,0.224798,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.600976,0.0,0.767003,0.0000
2,0.0,0.571631,0.186198,0.0,0.0,0.000000,0.0,0.799105,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0000
3,0.0,0.000000,0.534712,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.845034,0.000000,0.0,0.000000,0.0000
4,0.0,0.000000,0.534712,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.845034,0.000000,0.0,0.000000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27850,0.0,0.000000,0.534712,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.845034,0.000000,0.0,0.000000,0.0000
27851,0.0,0.553899,0.180423,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.8128
27852,0.0,0.553899,0.180423,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.8128
27853,0.0,0.000000,0.534712,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.845034,0.000000,0.0,0.000000,0.0000


In [ ]:
nn = NearestNeighbors(n_neighbors=5)
nn.fit(dtm_com.values)

NearestNeighbors()

In [ ]:
nn.kneighbors([dtm_com.iloc[2]])

(array([[0., 0., 0., 0., 0.]]), array([[22951, 19062, 23188,  1566,  5532]]))

In [ ]:
com_base.loc[com_base['user_session'] == '629a0485-36c9-4a80-9f02-4a96382c3059']

,event_type,category_code,brand,price,user_session
2,purchase,computers.components.hdd,kingston,66.67,629a0485-36c9-4a80-9f02-4a96382c3059


In [ ]:
com_base.iloc[[22951]]

,event_type,category_code,brand,price,user_session
22951,purchase,computers.components.hdd,samsung,204.87,30399143-8e68-43de-9d40-1ac965814690


In [ ]:
com_base.iloc[[19062]]

,event_type,category_code,brand,price,user_session
19062,purchase,computers.components.hdd,kingston,62.01,79fb2320-d81f-4c39-aa5e-6eb3eb87b7e8


In [ ]:
com_base.iloc[[23188]]

,event_type,category_code,brand,price,user_session
23188,purchase,computers.components.hdd,kingston,431.05,6f31018c-e5f2-400e-845a-202f32c522c8


In [ ]:
com_base.iloc[[1566]]

,event_type,category_code,brand,price,user_session
1566,purchase,computers.components.hdd,toshiba,250.66,c426776b-36db-40be-8f68-9767fca31cb7


In [ ]:
com_base.iloc[[5532]]

,event_type,category_code,brand,price,user_session
5532,purchase,computers.components.hdd,toshiba,44.49,dec9a27a-14bd-4c80-9183-9388a2dc5866


### computers baseline
- 유저간 추천

In [ ]:
tfidf = TfidfVectorizer(max_features = 100)

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_com_session = tfidf.fit_transform(com_base['user_session'])

dtm_com_session = pd.DataFrame(dtm_com_session.todense(), columns=tfidf.get_feature_names())
dtm_com_session

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,404c,4059,405d,40cf,40d7,40f1,4179,41b7,41d6,41ee,...,4fc9,4fcc,525b,91aaff3b5487,a0af,b439,b7396a89,bfe4,d095d5410e6b,eccfccbe
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
nn = NearestNeighbors(n_neighbors=5)
nn.fit(dtm_com_session.values)

NearestNeighbors()

In [ ]:
nn.kneighbors([dtm_com_session.iloc[2]])

(array([[0., 0., 0., 0., 0.]]), array([[18296, 18295, 18298, 18300, 18297]]))

In [ ]:
com_base.loc[com_base['user_session'] == '629a0485-36c9-4a80-9f02-4a96382c3059']

,event_type,category_code,brand,price,user_session
2,purchase,computers.components.hdd,kingston,66.67,629a0485-36c9-4a80-9f02-4a96382c3059


In [ ]:
com_base.iloc[[18296]]

,event_type,category_code,brand,price,user_session
18296,purchase,computers.desktop,lenovo,385.85,7b2d41c4-1357-4c43-bca2-510c5b37289c


In [ ]:
com_base.iloc[[18296]]

,event_type,category_code,brand,price,user_session
18296,purchase,computers.desktop,lenovo,385.85,7b2d41c4-1357-4c43-bca2-510c5b37289c


In [ ]:
com_base.iloc[[18298]]

,event_type,category_code,brand,price,user_session
18298,purchase,computers.notebook,acer,1106.82,afc38891-5452-4c47-9c66-cea345279b83


In [ ]:
com_base.iloc[[18300]]

,event_type,category_code,brand,price,user_session
18300,purchase,computers.notebook,hp,249.43,6c35b7d7-a20d-4d2f-878f-798aa7e65fdc


In [ ]:
com_base.iloc[[18297]]

,event_type,category_code,brand,price,user_session
18297,purchase,computers.peripherals.mouse,logitech,14.13,39fce100-afef-4bcc-8911-e85dab6f5fd4


### appliances baseline
- 제품간 추천

In [ ]:
appli_base = recommend.loc[(recommend['main_category'] == 'appliances')]

In [ ]:
appli_base.drop(columns=['event_time', 'product_id','category_id','main_category','subcategory'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
appli_base.reset_index(drop=True, inplace=True)

In [ ]:
appli_base.head()

,event_type,category_code,brand,price,user_id,user_session
0,purchase,appliances.environment.air_heater,oasis,28.03,548691404,b67cdbcb-b073-4271-b365-803c6fce53b0
1,purchase,appliances.kitchen.meat_grinder,philips,120.95,553819262,94b897fc-80c6-492c-a456-707398c7cc8e
2,purchase,appliances.kitchen.blender,panasonic,51.46,527681026,feb31d4f-ceb5-4710-8a8e-391abeaca106
3,purchase,appliances.kitchen.washer,lg,308.65,513419646,5e227e58-6117-419c-b45d-ac54be44716d
4,purchase,appliances.environment.vacuum,lg,189.26,512584306,510447fa-8dee-4027-a80a-a17dab9260af


In [ ]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_appli = tfidf.fit_transform(appli_base['category_code'])

dtm_appli = pd.DataFrame(dtm_appli.todense(), columns=tfidf.get_feature_names())
dtm_appli

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,air_conditioner,air_heater,appliances,blender,coffee_grinder,coffee_machine,dishwasher,environment,fan,grill,...,oven,personal,refrigerators,scales,sewing_machine,steam_cooker,toster,vacuum,washer,water_heater
0,0.0,0.859497,0.195004,0.000000,0.0,0.0,0.0,0.472480,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,0.0,0.000000,0.209067,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
2,0.0,0.000000,0.219490,0.922724,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
3,0.0,0.000000,0.324215,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.822077,0.0
4,0.0,0.000000,0.260655,0.000000,0.0,0.0,0.0,0.631546,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.730211,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74991,0.0,0.000000,0.228602,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
74992,0.0,0.000000,0.210041,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.977692,0.0,0.0,0.000000,0.000000,0.0
74993,0.0,0.000000,0.324215,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.822077,0.0
74994,0.0,0.000000,0.294971,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.85537,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0


In [ ]:
nn = NearestNeighbors(n_neighbors=5)
nn.fit(dtm_appli.values)

NearestNeighbors()

In [ ]:
nn.kneighbors([dtm_appli.iloc[2270]])

(array([[0., 0., 0., 0., 0.]]), array([[26988, 57937, 71675, 70806, 26987]]))

In [ ]:
appli_base.loc[appli_base['user_session'] == '4ff62316-20c5-4edf-a9c5-0e9bca818a18']

,event_type,category_code,brand,price,user_id,user_session
2270,purchase,appliances.kitchen.hood,hansa,102.68,547720935,4ff62316-20c5-4edf-a9c5-0e9bca818a18


In [ ]:
appli_base.iloc[[26988]]

,event_type,category_code,brand,price,user_id,user_session
26988,purchase,appliances.kitchen.hood,dauscher,95.21,515600682,b8924bc0-7245-40a9-a3c2-21f479158c1e


In [ ]:
appli_base.iloc[[57937]]

,event_type,category_code,brand,price,user_id,user_session
57937,purchase,appliances.kitchen.hood,gefest,57.66,517489779,940fb671-f7ed-45e7-b10e-10f003b4e658


In [ ]:
appli_base.iloc[[71675]]

,event_type,category_code,brand,price,user_id,user_session
71675,purchase,appliances.kitchen.hood,bosch,102.94,538413875,550c58de-1c2f-49f9-83a0-153a43130ffa


In [ ]:
appli_base.iloc[[70806]]

,event_type,category_code,brand,price,user_id,user_session
70806,purchase,appliances.kitchen.hood,turboair,40.85,558522747,11e1266b-facd-4788-b6eb-6e479da089cf


In [ ]:
appli_base.iloc[[26987]]

,event_type,category_code,brand,price,user_id,user_session
26987,purchase,appliances.kitchen.hood,dauscher,90.07,531283578,9c2c86f9-b2ad-4f7f-bd8a-0fc57138ae06


### appliances baseline
- 유저간 추천

In [ ]:
tfidf = TfidfVectorizer(max_features = 100)

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_appli_session = tfidf.fit_transform(appli_base['user_session'])

In [ ]:
dtm_appli_session = pd.DataFrame(dtm_appli_session.todense(), columns=tfidf.get_feature_names())
dtm_appli_session

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,4016,4048,4067,40b3,40f4,40fb,40fd,4135,4147,4165,...,4e7e,4eb7,4eba,4efb,4f3a,4f57,4f6c,4f9b,4fa2,4ff9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
nn = NearestNeighbors(n_neighbors=5)
nn.fit(dtm_appli_session.values)

NearestNeighbors()

In [ ]:
nn.kneighbors([dtm_appli_session.iloc[2270]])

(array([[0., 0., 0., 0., 0.]]), array([[49452, 49453, 49451, 49449, 49454]]))

In [ ]:
appli_base.loc[appli_base['user_session'] == '4ff62316-20c5-4edf-a9c5-0e9bca818a18']

,event_type,category_code,brand,price,user_id,user_session
2270,purchase,appliances.kitchen.hood,hansa,102.68,547720935,4ff62316-20c5-4edf-a9c5-0e9bca818a18


In [ ]:
appli_base.iloc[[49452]]

,event_type,category_code,brand,price,user_id,user_session
49452,purchase,appliances.kitchen.washer,samsung,334.6,518644283,c4f825de-0d53-4c67-bc00-d7476f509041


In [ ]:
appli_base.iloc[[49453]]

,event_type,category_code,brand,price,user_id,user_session
49453,purchase,appliances.kitchen.microwave,arg,56.6,523999245,7edca5cb-0b3a-4761-ae71-cbc04ca4e5a1


In [ ]:
appli_base.iloc[[49451]]

,event_type,category_code,brand,price,user_id,user_session
49451,purchase,appliances.environment.vacuum,philips,101.93,516252748,9f8e1994-af56-48df-b2b1-287436db0d73


In [ ]:
appli_base.iloc[[49449]]

,event_type,category_code,brand,price,user_id,user_session
49449,purchase,appliances.iron,vitek,28.06,543773886,df71ca64-4c2d-49b3-8f4d-44ea53e6a243


In [ ]:
appli_base.iloc[[49454]]

,event_type,category_code,brand,price,user_id,user_session
49454,purchase,appliances.kitchen.washer,samsung,593.79,521343725,41bd1ccb-3026-4400-81a0-2ee814354f51


### electronics baseline
- 제품간 추천

In [ ]:
elec_base = recommend.loc[(recommend['main_category'] == 'electronics')]

In [ ]:
elec_base.drop(columns=['event_time', 'event_type','product_id','category_id','main_category','subcategory'], inplace=True)
elec_base.reset_index(drop=True, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
elec_base.head()

,category_code,brand,price,user_id,user_session
0,electronics.smartphone,samsung,130.76,543272936,8187d148-3c41-46d4-b0c0-9c08cd9dc564
1,electronics.smartphone,apple,642.69,551377651,3c80f0d6-e9ec-4181-8c5c-837a30be2d68
2,electronics.audio.headphone,apple,189.91,524601178,2af9b570-0942-4dcd-8f25-4d84fba82553
3,electronics.audio.headphone,apple,161.98,551377651,3c80f0d6-e9ec-4181-8c5c-837a30be2d68
4,electronics.smartphone,apple,515.67,524325294,0b74a829-f9d7-4654-b5b0-35bc9822c238


In [ ]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_elec = tfidf.fit_transform(elec_base['category_code'])

In [ ]:
dtm_elec = pd.DataFrame(dtm_elec.todense(), columns=tfidf.get_feature_names())
dtm_elec

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,acoustic,audio,camera,clocks,electronics,headphone,microphone,music_tools,photo,piano,projector,smartphone,subwoofer,tablet,telephone,tv,video
0,0.0,0.000000,0.0,0.0,0.632581,0.000000,0.0,0.0,0.0,0.0,0.0,0.774494,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.632581,0.000000,0.0,0.0,0.0,0.0,0.0,0.774494,0.0,0.0,0.0,0.0,0.0
2,0.0,0.678288,0.0,0.0,0.195174,0.708401,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.678288,0.0,0.0,0.195174,0.708401,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.632581,0.000000,0.0,0.0,0.0,0.0,0.0,0.774494,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423023,0.0,0.000000,0.0,0.0,0.632581,0.000000,0.0,0.0,0.0,0.0,0.0,0.774494,0.0,0.0,0.0,0.0,0.0
423024,0.0,0.000000,0.0,0.0,0.632581,0.000000,0.0,0.0,0.0,0.0,0.0,0.774494,0.0,0.0,0.0,0.0,0.0
423025,0.0,0.000000,0.0,0.0,0.632581,0.000000,0.0,0.0,0.0,0.0,0.0,0.774494,0.0,0.0,0.0,0.0,0.0
423026,0.0,0.000000,0.0,0.0,0.632581,0.000000,0.0,0.0,0.0,0.0,0.0,0.774494,0.0,0.0,0.0,0.0,0.0


In [ ]:
nn = NearestNeighbors(n_neighbors=5)
nn.fit(dtm_elec.values)

NearestNeighbors()

In [ ]:
nn.kneighbors([dtm_elec.iloc[10]])

(array([[0., 0., 0., 0., 0.]]),
 array([[360163,  21670, 226526, 157434, 260435]]))

In [ ]:
elec_base.loc[elec_base['user_session'] == '0c990471-9407-4292-a6df-0cbc2f722a40']

,category_code,brand,price,user_id,user_session
10,electronics.audio.headphone,xiaomi,23.13,540557704,0c990471-9407-4292-a6df-0cbc2f722a40


In [ ]:
elec_base.iloc[[360163]]

,category_code,brand,price,user_id,user_session
360163,electronics.audio.headphone,xiaomi,43.5,513079004,beaacf93-9660-4eab-be29-456af011ef93


In [ ]:
elec_base.iloc[[21670]]

,category_code,brand,price,user_id,user_session
21670,electronics.audio.headphone,samsung,45.42,512790285,cf4695c3-b478-4044-bde5-54c42b67d53b


In [ ]:
elec_base.iloc[[226526]]

,category_code,brand,price,user_id,user_session
226526,electronics.audio.headphone,apple,160.62,516057735,c2e90167-31c8-4c24-8fc7-49d0a449cb5b


In [ ]:
elec_base.iloc[[157434]]

,category_code,brand,price,user_id,user_session
157434,electronics.audio.headphone,huawei,38.97,519018305,4923005e-9c55-4789-ae2a-5d911a3bfcaa


In [ ]:
elec_base.iloc[[260435]]

,category_code,brand,price,user_id,user_session
260435,electronics.audio.headphone,apple,160.62,516228144,a7d973cd-2a10-484f-9d32-73c7c2918635


### electronics baseline
- 유저간 추천

In [ ]:
tfidf = TfidfVectorizer(max_features = 100)

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_elec_session = tfidf.fit_transform(elec_base['user_session'])

In [ ]:
dtm_elec_session = pd.DataFrame(dtm_elec_session.todense(), columns=tfidf.get_feature_names())
dtm_elec_session

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,4005,4008,4079,40fc,4103,4163,416d,419f,41cf,4244,...,4e7d,4e8c,4eb3,4ec2,4ee0,4efe,4f8d,4f90,4fa6,4fd4
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
423024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
423025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
423026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
nn = NearestNeighbors(n_neighbors=5)
nn.fit(dtm_elec_session.values)

NearestNeighbors()

In [ ]:
nn.kneighbors([dtm_elec_session.iloc[10]])

(array([[0., 0., 0., 0., 0.]]),
 array([[279575, 279574, 279577, 279578, 279576]]))

In [ ]:
elec_base.loc[(elec_base['user_session'] == '0c990471-9407-4292-a6df-0cbc2f722a40')]

,category_code,brand,price,user_id,user_session
10,electronics.audio.headphone,xiaomi,23.13,540557704,0c990471-9407-4292-a6df-0cbc2f722a40


In [ ]:
elec_base.iloc[[279575]]

,category_code,brand,price,user_id,user_session
279575,electronics.smartphone,oppo,257.38,560370787,1d5b25ec-2182-412c-8d4d-d66d27df8fbd


In [ ]:
elec_base.iloc[[279574]]

,category_code,brand,price,user_id,user_session
279574,electronics.smartphone,samsung,217.04,537726404,9d2fe03b-673d-44b6-b1ff-ac479c2cef99


In [ ]:
elec_base.iloc[[279577]]

,category_code,brand,price,user_id,user_session
279577,electronics.smartphone,apple,533.35,562337268,fefa6e1b-6866-4fff-983d-5759a18ce048


In [ ]:
elec_base.iloc[[279578]]

,category_code,brand,price,user_id,user_session
279578,electronics.audio.headphone,apple,190.34,520579186,57173626-c824-411b-89b5-4609ebcef3b1


In [ ]:
elec_base.iloc[[279576]]

,category_code,brand,price,user_id,user_session
279576,electronics.smartphone,samsung,370.6,562458518,15773585-45c4-4a44-ac0f-148db3a1e0fb


## Cosine 유사도 모델

### computers 제품간 추천

In [ ]:
# 한번이하로 구매한 고객 중 컴퓨터 제품을 구매한 고객 분리
com_once = RFM_without_view.loc[(RFM_without_view['Frequency'] < 2) & (RFM_without_view['event_type'] == 'purchase') & (RFM_without_view['main_category'] == 'computers')]

In [ ]:
com_once.drop(columns = ['event_type','product_id', 'main_category', 'subcategory','Frequency','RFM_label'], inplace=True)
com_once = com_once.reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
com_once.head()

,category_code,brand,price,user_id,user_session
0,computers.peripherals.printer,epson,116.73,513489079,63de1120-9d48-45d0-b024-500d1b95d71f
1,computers.components.hdd,kingston,66.67,523117953,629a0485-36c9-4a80-9f02-4a96382c3059
2,computers.notebook,acer,532.81,549790258,56272bb8-9f35-4412-9f15-624c8ecda0f1
3,computers.notebook,hp,285.51,555467053,8d2582ff-8031-4b06-819b-a653eb1cb832
4,computers.notebook,hp,285.51,521162064,5622db13-66d1-4167-8bfa-b4f184aed29a


In [ ]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_tfidf = tfidf.fit_transform(com_once['category_code'])

In [ ]:
dtm_tfidf_df= pd.DataFrame(dtm_tfidf.todense(), columns=tfidf.get_feature_names_out())
dtm_tfidf_df

,camera,components,computers,cooler,cpu,desktop,ebooks,hdd,keyboard,memory,monitor,motherboard,mouse,notebook,peripherals,power_supply,printer,videocards
0,0.0,0.00000,0.237006,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.613976,0.0,0.752903,0.0
1,0.0,0.59821,0.184131,0.0,0.0,0.000000,0.0,0.779898,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
2,0.0,0.00000,0.534484,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.845179,0.000000,0.0,0.000000,0.0
3,0.0,0.00000,0.534484,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.845179,0.000000,0.0,0.000000,0.0
4,0.0,0.00000,0.534484,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.845179,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,0.0,0.00000,0.534484,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.845179,0.000000,0.0,0.000000,0.0
9121,0.0,0.00000,0.301918,0.0,0.0,0.953334,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
9122,0.0,0.00000,0.191205,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.847403,0.000000,0.495327,0.0,0.000000,0.0
9123,0.0,0.00000,0.534484,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.845179,0.000000,0.0,0.000000,0.0


In [ ]:
cosine_matrix = cosine_similarity(dtm_tfidf, dtm_tfidf)

In [ ]:
np.round(cosine_matrix, 4)

array([[1.    , 0.0445, 0.1253, ..., 0.3539, 0.1253, 0.1253],
       [0.0445, 1.    , 0.0996, ..., 0.0369, 0.0996, 0.0996],
       [0.1253, 0.0996, 1.    , ..., 0.104 , 1.    , 1.    ],
       ...,
       [0.3539, 0.0369, 0.104 , ..., 1.    , 0.104 , 0.104 ],
       [0.1253, 0.0996, 1.    , ..., 0.104 , 1.    , 1.    ],
       [0.1253, 0.0996, 1.    , ..., 0.104 , 1.    , 1.    ]])

In [ ]:
# 유저 아이디에 인덱스를 지정하여 유저 아이디를 검색하였을 때 유저가 구매한 제품과 유사한 제품을 추천
user = {}
for i, c in enumerate(com_once['user_id']): 
  user[i] = c

user_id = {}
for i, c in user.items():
  user_id[c] = i

In [ ]:
idx = user_id[523117953]
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 유저들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:5] # 상위 5개의 인덱스와 유사도를 추출 

[(29, 1.0000000000000002),
 (48, 1.0000000000000002),
 (52, 1.0000000000000002),
 (64, 1.0000000000000002),
 (71, 1.0000000000000002)]

In [ ]:
com_once.loc[com_once['user_id'] == 523117953]

,category_code,brand,price,user_id,user_session
1,computers.components.hdd,kingston,66.67,523117953,629a0485-36c9-4a80-9f02-4a96382c3059


In [ ]:
com_once.iloc[[29]]

,category_code,brand,price,user_id,user_session
29,computers.components.hdd,apacer,24.2,517465372,c98f69e7-a2a7-4595-8fa5-576ca4f5106d


In [ ]:
com_once.iloc[[48]]

,category_code,brand,price,user_id,user_session
48,computers.components.hdd,kingston,38.32,517010638,c67ef2b3-3b51-48e1-9c10-298ed8cc5594


In [ ]:
com_once.iloc[[52]]

,category_code,brand,price,user_id,user_session
52,computers.components.hdd,NaN,27.74,530849675,28080e96-d254-40f0-8cb8-b4df5a8bc192


In [ ]:
com_once.iloc[[64]]

,category_code,brand,price,user_id,user_session
64,computers.components.hdd,NaN,62.36,520118791,2a156b42-c6b9-470e-816b-377d9d2a4294


In [ ]:
com_once.iloc[[71]]

,category_code,brand,price,user_id,user_session
71,computers.components.hdd,toshiba,39.02,531895455,98fc6737-15ac-496a-8b4f-6e4c6dd248cb


### computers 유저간 추천

In [ ]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_tfidf_user = tfidf.fit_transform(com_once['user_session'])

In [ ]:
dtm_tfidf_user_df= pd.DataFrame(dtm_tfidf_user.todense(), columns=tfidf.get_feature_names_out())
dtm_tfidf_user_df

,0002,00025dab,0002c5ea,0005,0006bf97f732,0007ab79ced4,0011,0012d784,001312a0142e,00153b60a6e5,...,ffeb,ffed,ffee,fff6,fff60634,fff661e40162,fff9,fffa,fffac901e3c9,fffc
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
cosine_matrix = cosine_similarity(dtm_tfidf_user, dtm_tfidf_user)

In [ ]:
np.round(cosine_matrix, 4)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
# 유저 아이디를 인덱스화하여 유저 아이디를 검색하였을 때 유저와 비슷한 다른 유저가 구매한 다른 제품을 추천
user = {}
for i, c in enumerate(com_once['user_id']): 
  user[i] = c

user_id = {}
for i, c in user.items():
  user_id[c] = i

In [ ]:
idx_category = user_id[523117953]
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx_category]) if i != idx_category] # 자기 자신을 제외한 유저들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:5] # 상위 5개의 인덱스와 유사도를 추출 

[(3284, 0.1915772911280588),
 (549, 0.17340575844222422),
 (388, 0.17235175931248922),
 (4309, 0.17235175931248922),
 (0, 0.0)]

In [ ]:
com_once.loc[com_once['user_session'] == '629a0485-36c9-4a80-9f02-4a96382c3059']

,category_code,brand,price,user_id,user_session
1,computers.components.hdd,kingston,66.67,523117953,629a0485-36c9-4a80-9f02-4a96382c3059


In [ ]:
com_once.iloc[[3284]]

,category_code,brand,price,user_id,user_session
3284,computers.peripherals.printer,epson,244.28,527550767,efd66ffb-9f02-4313-979f-dba38f8c3464


In [ ]:
com_once.iloc[[549]]

,category_code,brand,price,user_id,user_session
549,computers.notebook,hp,320.35,551478118,9c664cb4-9311-4a80-bd17-fca008e49c98


In [ ]:
com_once.iloc[[388]]

,category_code,brand,price,user_id,user_session
388,computers.components.hdd,transcend,45.05,548837228,165a2320-9850-4a80-845b-1b1fcbc1143d


In [ ]:
com_once.iloc[[4309]]

,category_code,brand,price,user_id,user_session
4309,computers.notebook,asus,540.53,559683462,70d3de0c-ce16-4a80-b6b0-e69b3393af9a


### appliances 제품간 추천

In [ ]:
# 가전제품을 구매한 고객 중 누적구매횟수가 1회 이하인 고객을 분리
appli_once = RFM_without_view.loc[(RFM_without_view['Frequency'] < 2) & (RFM_without_view['event_type'] == 'purchase') & (RFM_without_view['main_category'] == 'appliances')]

In [ ]:
appli_once.drop(columns=['event_type','product_id','main_category','subcategory','Frequency', 'RFM_label'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
appli_once = appli_once.reset_index(drop=True)

In [ ]:
appli_once.head()

,category_code,brand,price,user_id,user_session
0,appliances.environment.air_heater,oasis,28.03,548691404,b67cdbcb-b073-4271-b365-803c6fce53b0
1,appliances.kitchen.meat_grinder,philips,120.95,553819262,94b897fc-80c6-492c-a456-707398c7cc8e
2,appliances.kitchen.washer,lg,308.65,513419646,5e227e58-6117-419c-b45d-ac54be44716d
3,appliances.kitchen.refrigerators,NaN,204.33,516426855,788ec8fe-f198-4484-b911-ba5841c55c68
4,appliances.personal.hair_cutter,moser,47.88,514018346,35323009-6410-4412-930e-e7855384a5da


In [ ]:
appli_once.shape

(31936, 5)

In [ ]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_appli = tfidf.fit_transform(appli_once['category_code'])

In [ ]:
dtm_appli_df= pd.DataFrame(dtm_appli.todense(), columns=tfidf.get_feature_names_out())
dtm_appli_df

,air_conditioner,air_heater,appliances,blender,coffee_grinder,coffee_machine,dishwasher,environment,fan,grill,...,oven,personal,refrigerators,scales,sewing_machine,steam_cooker,toster,vacuum,washer,water_heater
0,0.0,0.862929,0.196043,0.0,0.0,0.0,0.0,0.465747,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
1,0.0,0.000000,0.210327,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
2,0.0,0.000000,0.317425,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.82639,0.0
3,0.0,0.000000,0.293136,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.85416,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
4,0.0,0.000000,0.145957,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.631627,0.00000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31931,0.0,0.000000,0.185239,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
31932,0.0,0.000000,0.317425,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.82639,0.0
31933,0.0,0.000000,0.226555,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
31934,0.0,0.000000,0.212498,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.977161,0.0,0.0,0.0,0.00000,0.0


In [ ]:
cosine_matrix = cosine_similarity(dtm_appli, dtm_appli)

In [ ]:
np.round(cosine_matrix, 4)

array([[1.    , 0.0412, 0.0622, ..., 0.0444, 0.0417, 0.0622],
       [0.0412, 1.    , 0.2101, ..., 0.15  , 0.0447, 0.2101],
       [0.0622, 0.2101, 1.    , ..., 0.2263, 0.0675, 1.    ],
       ...,
       [0.0444, 0.15  , 0.2263, ..., 1.    , 0.0481, 0.2263],
       [0.0417, 0.0447, 0.0675, ..., 0.0481, 1.    , 0.0675],
       [0.0622, 0.2101, 1.    , ..., 0.2263, 0.0675, 1.    ]])

In [ ]:
user = {}
for i, c in enumerate(appli_once['user_id']): 
  user[i] = c

user_id = {}
for i, c in user.items():
  user_id[c] = i

In [ ]:
idx = user_id[547720935]
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 유저들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:5] # 상위 5개의 인덱스와 유사도를 추출 

[(5, 1.0), (124, 1.0), (182, 1.0), (184, 1.0), (220, 1.0)]

In [ ]:
appli_once.loc[appli_once['user_id'] == 547720935]

,category_code,brand,price,user_id,user_session
1018,appliances.kitchen.hood,hansa,102.68,547720935,4ff62316-20c5-4edf-a9c5-0e9bca818a18


In [ ]:
appli_once.iloc[[5]]

,category_code,brand,price,user_id,user_session
5,appliances.kitchen.hood,artel,61.24,518431897,d244ccce-4dbb-4ae6-af48-98eb36069e19


In [ ]:
appli_once.iloc[[124]]

,product_id,category_code,brand,price,user_id
124,2400103,appliances.kitchen.hood,turboair,51.46,540583065


In [ ]:
appli_once.iloc[[182]]

,product_id,category_code,brand,price,user_id
182,2400930,appliances.kitchen.hood,hansa,205.67,555357759


In [ ]:
appli_once.iloc[[184]]

,product_id,category_code,brand,price,user_id
184,2400953,appliances.kitchen.hood,gorenje,154.18,550314548


In [ ]:
appli_once.iloc[[220]]

,product_id,category_code,brand,price,user_id
220,2402570,appliances.kitchen.hood,bosch,102.94,555514854


### appliances 유저간 추천

In [ ]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_appli_user = tfidf.fit_transform(appli_once['user_session'])

In [ ]:
cosine_matrix = cosine_similarity(dtm_appli_user, dtm_appli_user)

In [ ]:
np.round(cosine_matrix, 4)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
user = {}
for i, c in enumerate(appli_once['user_id']): 
  user[i] = c

user_id = {}
for i, c in user.items():
  user_id[c] = i

In [ ]:
idx = user_id[547720935]
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 유저들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:5] # 상위 5개의 인덱스와 유사도를 추출 

[(23095, 0.2096893344877848),
 (12106, 0.18078621866047573),
 (5232, 0.18013754090842443),
 (5233, 0.18013754090842443),
 (5983, 0.17730991497607076)]

In [ ]:
appli_once.loc[appli_once['user_id'] == 547720935]

,category_code,brand,price,user_id,user_session
1018,appliances.kitchen.hood,hansa,102.68,547720935,4ff62316-20c5-4edf-a9c5-0e9bca818a18


In [ ]:
appli_once.iloc[[23095]]

,product_id,category_code,brand,price,user_id,user_session
23095,3901044,appliances.environment.water_heater,aqua,51.22,528137982,b345ec1c-20c5-42fe-836c-b901bbfd1668


In [ ]:
appli_once.iloc[[12106]]

,product_id,category_code,brand,price,user_id,user_session
12106,3200157,appliances.kitchen.meat_grinder,bosch,149.27,519289318,1937dc12-42e1-4f79-a9c5-7f4ea92c3011


In [ ]:
appli_once.iloc[[5232]]

,product_id,category_code,brand,price,user_id,user_session
5232,3800548,appliances.iron,vitek,28.29,514528702,5c59ea44-2773-4969-a9c5-1217a6cb6d36


In [ ]:
appli_once.iloc[[5233]]

,product_id,category_code,brand,price,user_id,user_session
5233,3800548,appliances.iron,vitek,28.29,514528702,5c59ea44-2773-4969-a9c5-1217a6cb6d36


In [ ]:
appli_once.iloc[[5983]]

,category_code,brand,price,user_id,user_session
5983,appliances.kitchen.hood,gorenje,141.29,513669164,cdd6e5f3-198f-4a27-a9c5-f509eb9edd0b


### electronics 제품간 추천
- segment5에 대한 추천

In [ ]:
# 전자제품을 구매하고 누적 구매횟수가 1회이하로 구매한 고객 중 segment5에 속하는 고객군 분리
elec_once = RFM_without_view.loc[(RFM_without_view['Frequency'] < 2) & (RFM_without_view['event_type'] == 'purchase') & (RFM_without_view['main_category'] == 'electronics') & (RFM_without_view['RFM_label'] == 'segment5')]

In [ ]:
elec_once['RFM_label'].value_counts()

segment4    64996
segment3    44152
segment5    22159
segment2     7114
Name: RFM_label, dtype: int64

In [ ]:
elec_once.drop(columns=['event_type','product_id', 'main_category','subcategory','Frequency','RFM_label'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
elec_once.reset_index(drop=True, inplace=True)

In [ ]:
elec_once.head()

,category_code,brand,price,user_id,user_session
0,electronics.audio.headphone,apple,189.91,524601178,2af9b570-0942-4dcd-8f25-4d84fba82553
1,electronics.audio.headphone,xiaomi,23.13,540557704,0c990471-9407-4292-a6df-0cbc2f722a40
2,electronics.smartphone,huawei,111.82,512812487,d3ef047f-9476-4b8d-a126-b483c55de653
3,electronics.smartphone,huawei,177.20,518962372,718c8069-2782-4585-acad-6747f6f2f79d
4,electronics.smartphone,xiaomi,185.71,552439521,80ccabf1-39b6-421f-b452-4bf9e37805d6


In [ ]:
elec_once.shape

(22159, 5)

In [ ]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_elec = tfidf.fit_transform(elec_once['category_code'])

In [ ]:
cosine_matrix = cosine_similarity(dtm_elec, dtm_elec)

In [ ]:
np.round(cosine_matrix, 4)

array([[1.    , 1.    , 0.1473, ..., 0.1473, 0.1473, 0.0675],
       [1.    , 1.    , 0.1473, ..., 0.1473, 0.1473, 0.0675],
       [0.1473, 0.1473, 1.    , ..., 1.    , 1.    , 0.1474],
       ...,
       [0.1473, 0.1473, 1.    , ..., 1.    , 1.    , 0.1474],
       [0.1473, 0.1473, 1.    , ..., 1.    , 1.    , 0.1474],
       [0.0675, 0.0675, 0.1474, ..., 0.1474, 0.1474, 1.    ]])

In [ ]:
user = {}
for i, c in enumerate(elec_once['user_id']): 
  user[i] = c

user_id = {}
for i, c in user.items():
  user_id[c] = i

In [ ]:
idx = user_id[540557704]
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 유저들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:5] # 상위 5개의 인덱스와 유사도를 추출 

[(0, 1.0), (31, 1.0), (37, 1.0), (57, 1.0), (74, 1.0)]

In [ ]:
elec_once.loc[elec_once['user_id'] == 540557704]

,category_code,brand,price,user_id,user_session
1,electronics.audio.headphone,xiaomi,23.13,540557704,0c990471-9407-4292-a6df-0cbc2f722a40


In [ ]:
elec_once.iloc[[0]]

,category_code,brand,price,user_id,user_session
0,electronics.audio.headphone,apple,189.91,524601178,2af9b570-0942-4dcd-8f25-4d84fba82553


In [ ]:
elec_once.iloc[[31]]

,category_code,brand,price,user_id,user_session
31,electronics.audio.headphone,apple,161.98,543650701,4c9b3b0d-9511-481a-ba0b-b5350a7cf0b4


In [ ]:
elec_once.iloc[[37]]

,category_code,brand,price,user_id,user_session
37,electronics.audio.headphone,marshall,89.83,517163214,af635016-1f2d-4b33-830a-09743991c812


In [ ]:
elec_once.iloc[[57]]

,category_code,brand,price,user_id,user_session
57,electronics.audio.headphone,xiaomi,22.39,514536402,4652dea0-b6a0-4dd7-8a0d-8c1323e4d461


In [ ]:
elec_once.iloc[[74]]

,category_code,brand,price,user_id,user_session
74,electronics.audio.headphone,apple,188.73,539281255,0615b21d-900c-4df2-8409-b8348011b168


### electronics 유저간 추천
- segment5 에 대한 추천

In [ ]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_elec_user = tfidf.fit_transform(elec_once['user_session'])

In [ ]:
cosine_matrix = cosine_similarity(dtm_elec_user, dtm_elec_user)

In [ ]:
np.round(cosine_matrix, 4)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
user = {}
for i, c in enumerate(elec_once['user_id']): 
  user[i] = c

user_id = {}
for i, c in user.items():
  user_id[c] = i

In [ ]:
idx = user_id[540557704]
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 유저들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:5] # 상위 5개의 인덱스와 유사도를 추출 

[(8152, 0.19936300100317822),
 (21390, 0.19603531121655107),
 (21421, 0.19603531121655107),
 (18505, 0.16982092407133242),
 (1389, 0.16907715071482754)]

In [ ]:
elec_once.loc[elec_once['user_id'] == 540557704]

,category_code,brand,price,user_id,user_session
1,electronics.audio.headphone,xiaomi,23.13,540557704,0c990471-9407-4292-a6df-0cbc2f722a40


In [ ]:
elec_once.iloc[[8152]]

,category_code,brand,price,user_id,user_session
8152,electronics.smartphone,huawei,109.36,556943187,58f8c518-29f8-4e00-a6df-af6682f173b6


In [ ]:
elec_once.iloc[[21390]]

,category_code,brand,price,user_id,user_session
21390,electronics.audio.headphone,xiaomi,6.18,536671878,f9cec814-9407-4921-9e24-6cad47f47e9a


In [ ]:
elec_once.iloc[[21421]]

,category_code,brand,price,user_id,user_session
21421,electronics.audio.headphone,xiaomi,5.79,536671878,f9cec814-9407-4921-9e24-6cad47f47e9a


In [ ]:
elec_once.iloc[[18505]]

,category_code,brand,price,user_id,user_session
18505,electronics.audio.headphone,apple,161.87,534776631,513d38f5-ea73-4292-8fcb-eaada6ab10e9


In [ ]:
elec_once.iloc[[1389]]

,category_code,brand,price,user_id,user_session
1389,electronics.smartphone,xiaomi,100.12,527259774,548ae7ce-068c-4292-938d-67c4fc2e4294
